In [ ]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import gmean

In [ ]:
core = 'HP'
#core = 'A76'
wblock = 8
bench = ['intNN', 'streamvbyte', 'cartoon', 'sqnrm2', 'amax', 'gemv', 'gemm', 'fft', 'conv', 'median',
         'hist', 'img_integral', 'canny', 'erode']
bench_paper = ['intNN', 'streamvbyte', 'cartoon', 'sqnrm2', 'amax', 'gemm', 'fft', 'conv', 'median',
               'img_integral', 'canny', 'erode']
modes = ['Original', 'Packing', 'Fusing', 'PackingPenalty', 'FusingPenalty']

In [ ]:
th = 1.10

In [ ]:
# Create best config table.

df = pd.read_csv('./tables/results_diffenergy_time.csv', header=0)
df = df.set_index(['Benchmark', 'CPU', 'Width Block', 'Mode', 'SIMD FU', 'Metric'])
df = df.xs((core, wblock), level=[1, 2])
df = df.loc(axis=0)[:, :, :, ['Total', 'Time']]
df = df.unstack().loc(axis=1)['Value']
s_th = df[df['Time'] <= th].groupby(['Benchmark', 'Mode'])['Total'].idxmin()
s_best_time = df.groupby(['Benchmark', 'Mode'])['Time'].idxmin()
df = df.loc[s_th.combine_first(s_best_time)]
df = df.reset_index(level=2)
df = df.unstack().reorder_levels(['Mode', 'Metric'], axis=1).sort_index(axis=1)
df

In [ ]:
# Get best config table (for thesis and paper).

def get_best_config(df, bench):
    df = df.copy()
    
    df = df.reindex(index=bench)
    df.loc(axis=0)['average'] = gmean(df)
    df.loc(axis=1)[:, 'SIMD FU'] = df.loc(axis=1)[:, 'SIMD FU'].astype(int).astype(str)
    df.loc['average', (slice(None), 'SIMD FU')] = '---'
    df.loc(axis=1)[:, 'Total'] = 100 * (1 - df.loc(axis=1)[:, 'Total'])
    df = df.reindex(columns=modes, level=0)
    df = df.reindex(columns=['SIMD FU', 'Total', 'Time'], level=1)
    df.loc(axis=1)[:, 'Time'] = df.loc(axis=1)[:, 'Time'].round(2)
    df.loc(axis=1)[:, 'Total'] = df.loc(axis=1)[:, 'Total'].round(1)
    df = df.rename(columns={'Total':'Save [%]', 'SIMD FU':'Units'}, level=1)
    
    return df

df_thesis = get_best_config(df, bench)
df_paper = get_best_config(df, bench_paper)

In [ ]:
# Save tables to .csv and .tex

# Thesis
df_thesis.to_latex('./tables/best_{}_thesis.tex'.format(core.lower()), index_names=False,
                  column_format=(1 + len(df_thesis.columns))*'c',
                  multicolumn_format='c')
df_thesis.to_csv('./tables/best_{}_thesis.csv'.format(core.lower()))

# Paper
df_paper.to_latex('./tables/best_{}_paper.tex'.format(core.lower()), index_names=False,
                  column_format=(1 + len(df_paper.columns))*'c',
                  multicolumn_format='c')
df_paper.to_csv('./tables/best_{}_paper.csv'.format(core.lower()))